In [1]:
import numpy as np
# from activfunc import relu, ..., activfunc_dict # relu is a class, activfunc is a file
# from lossfunc import mse # mse is a class, lossfunc is a file
# or just the dictionaries will do?

In [4]:
class neuralnetwork:
    
    def __init__(self):
        self.W = []
        self.Z = []
        self.Y = []
        self.Wgrad = None
        self.nlayers = 0
        self.activations = []
        self.prev_layer_neurons = None
        
    def add_layer(self, num_neurons, activation = 'relu'):
        ## TODO: add bias
        if self.nlayers == 0:
            self.W.append(None)
        else:
            weights = np.random.rand(num_neurons, self.prev_layer_neurons)
            self.W.append(weights)
        self.activations.append(activation.lower())
        self.prev_layer_neurons = num_neurons
        self.nlayers += 1
    
    def forward(self, inputs):
        
        current_x = inputs
        self.Y.append(inputs)
        for k in range(nlayers):
            z = (np.matmul(self.W[k], current_x.T)).T
            Z.append(z)
            
            sigma = activfunc_dict[self.activations[k]]
            y = sigma.forward(z)
            self.Y.append(y)
            current_x = y
        return y
        
    def backward(self, op_gradient):
        # Assuming op_gradient = row vector (convention)
        # diag of Jacobian (scalar activation)
        current_grad = op_gradient
        for k in range(nlayers, -1, -1): # opvec = Y[nlayers]
            sigma = activfunc_dict[self.activations[k - 1]]
            gradDz = np.multiply(current_grad, sigma.backward(self.Z[k - 1])) 
            # if local grad = matrix (Jacobian), use matmul else multiply
            gradzw = np.matmul(np.ones(np.shape(W[k - 1])[0]), self.Y[k - 1])
            self.Wgrad[k - 1] = np.matmul(gradDz, gradzw)
            current_grad = np.matmul(gradDz, self.W[k - 1])
        
    def train_network(self, X_train, Y_train, method = 'SGD', loss_function, eta = 0.001):
        
        input_size = np.shape(X_train)[1]
        layer1_size = np.shape(self.W[1])[1]
        self.W[0] = np.random.rand(layer1_size, input_size)
        self.Wgrad = np.zeros(np.shape(self.W))
        lossfunc = lossfunc_dict[loss_function.lower()]
        
        prev_loss = 0
        while True:
            permut = np.random.permutation(len(Y_train))
            X_train = X_train[permut]
            Y_train = Y_train
            for idx, x in enumerate(X_train):
                
                Y_pred = self.forward(x)
                loss = lossfunc(Y_pred, Y_train[idx])
                
                if abs(prev_loss - loss) < 0.01: 
                    break # stopping condition 
                    
                op_gradient = lossfunc.gradient(Y_pred, Y_train[idx])
                self.backward(op_gradient)
                self.W -= eta * self.Wgrad # SGD update
                prev_loss = loss
        
    

In [ ]:
NN = network(8)
NN.add_layer(12, activation = 'relu')